In [1]:
from pandas_datareader import data
import pandas_datareader as pdr
import pandas as pd
import numpy as np
#import talib as ta
import pandas_ta as ta
import datetime
import requests

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')


# Retrieve Trading data from houbi , 4hours

In [2]:
ticker = 'apeusdt'
base = 'https://api.huobi.pro'
url = '/market/history/kline?period=4hour&size=2000&symbol=ethusdt'
final_url = base + url

reqs = requests.get(final_url)

df = pd.DataFrame(reqs.json()['data'])
df["Date"] = pd.to_datetime(df["id"], unit='s', origin='unix')
df["Date"] = df["Date"].dt.strftime("%m/%d/%Y %H:%M:%S")
df.drop('id',axis = 1, inplace = True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
df.rename(columns={'high':'High','low':'Low','open':'Open','close':'Close','vol':'Volume'},inplace=True)
df['Adj Close']= df['Close']
#df = df[['High','Low','Open','Close','Volume','Adj Close']]
df.sort_index(inplace = True)



# Generate Super Tranding & Buy Sell Signal

In [3]:
df["sup"] = ta.supertrend(high = df["High"], low = df["Low"], close = df["Close"], period = 10 , mutiplier = 3)["SUPERT_7_3.0"]


# Generate MACD

In [26]:
ta.macd(df['Close'],fast =12 , slow =26 ,signal =9 )

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,
2021-06-15 20:00:00,NaN,NaN,NaN
2021-06-16 00:00:00,NaN,NaN,NaN
2021-06-16 04:00:00,NaN,NaN,NaN
2021-06-16 08:00:00,NaN,NaN,NaN
2021-06-16 12:00:00,NaN,NaN,NaN
...,...,...,...
2022-05-14 08:00:00,-90.423998,15.494232,-105.918230
2022-05-14 12:00:00,-90.557512,12.288574,-102.846087
2022-05-14 16:00:00,-87.056676,12.631529,-99.688204


In [4]:
df["Buy Signal"] = 0
df["Sell Signal"] = 0 
n = 9 
for i in range (n,len(df)):
    if df["Close"][i-1] <= df["sup"][i-1] and df["Close"][i] > df["sup"][i]:
        df["Buy Signal"][i] = 1 
    if df["Close"][i-1] >= df["sup"][i-1] and df["Close"][i] < df["sup"][i]:
        df["Sell Signal"][i] = 1
 
df['Budget'] = 100.00


# Back testing 

In [6]:
Buy_qty = 1
buying = False
cost = 0 

for x in range(1,len(df)):
    if df['Buy Signal'][x] == 1 and buying == False :
        cost = df['Budget'][x-1] * Buy_qty
        df['Budget'][x] = df['Budget'][x-1] * (1- Buy_qty)
        buy_price = df['Close'][x]
        buying = True

    else :
        df['Budget'][x] = df['Budget'][x-1]
    
    if df['Sell Signal'][x] == 1 and buying == True:
        temp_budget = df['Budget'][x-1]+ (cost * (df['Close'][x]- buy_price)/buy_price) +cost
        df['Budget'][x]=round(temp_budget,2)
        #print()
        buying = False





In [18]:
df[(df['Buy Signal']==1) | (df['Sell Signal']==1)]

,Open,Close,Low,High,amount,Volume,count,Adj Close,sup,Buy Signal,Sell Signal,Budget
Date,,,,,,,,,,,,
2021-06-18 12:00:00,2319.50,2215.34,2211.31,2325.94,101682.098161,2.297659e+08,192362,2215.34,2487.807679,0,1,100.00
2021-06-28 04:00:00,1967.50,2048.56,1964.51,2055.78,98148.812149,1.970201e+08,137149,2048.56,1778.407244,1,0,0.00
2021-07-08 04:00:00,2254.06,2182.69,2165.00,2254.06,81326.008283,1.803752e+08,144090,2182.69,2415.928930,0,1,106.55
2021-07-21 08:00:00,1875.14,1906.35,1853.24,1913.20,81099.924207,1.525966e+08,120943,1906.35,1713.171653,1,0,0.00
2021-07-27 00:00:00,2227.88,2172.78,2151.51,2240.86,81078.900120,1.775229e+08,168298,2172.78,2446.520560,0,1,121.44
2021-07-30 20:00:00,2378.76,2461.85,2374.84,2470.64,86594.172251,2.106133e+08,185214,2461.85,2231.196243,1,0,0.00
2021-08-17 20:00:00,3117.80,3012.21,2990.00,3127.31,107554.796434,3.269605e+08,204533,3012.21,3350.967567,0,1,148.59
2021-08-20 12:00:00,3204.66,3283.50,3203.50,3300.92,65788.013647,2.147773e+08,129395,3283.50,3028.664375,1,0,0.00
2021-08-25 08:00:00,3183.18,3139.25,3080.00,3187.11,61887.095886,1.933160e+08,138279,3139.25,3350.552293,0,1,142.06
